In [ ]:
#############################################################################################################
## Estimate average temperature based on X,Y point and date information
## Coded specifically towards data being provided by researcher, MK
## Can easily be adapted to other input data. Changes would probably be need to made to the date logic
## 
## Currently configured to run cell-by-cell in a Notebook. Should be configured properly with functions and main

In [ ]:
import arcpy, os, time
from datetime import datetime

arcpy.env.workspace = r'in_memory'
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension('Spatial')

In [ ]:
## Function to find nearest date to observation based on a list of temperature raster dates
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [ ]:
## Set spatial reference based on input observations layer
arcpy.env.outputCoordinateSystem = arcpy.Describe(observations_layer).spatialReference

## Copy observations layer to in_memory workspace
observations = arcpy.CopyFeatures_management(observations_layer, "observations")

## Initialize local variables
count = 0
datelist = []
abs_raster_dict = {}
proc_layer = None
proc_datetime = str(datetime.now().strftime("%Y%m%d_%H%M"))
procname = "proc_layer_temp_" + proc_datetime
procflag = False

In [ ]:
nc_temp= arcpy.NetCDFFileProperties(in_nc_temp)
dim_temp = nc_temp.getDimensions()
top = nc_temp.getDimensionSize("time")

## Populate list of temperature anomoly raster dates
for i in range(0,top):
    try:
        datelist.append(datetime.strptime(nc_temp.getDimensionValue("time",i), '%m/%d/%Y %I:%M:%S %p'))
    except ValueError:
        datelist.append(datetime.strptime(nc_temp.getDimensionValue("time",i), '%m/%d/%Y'))

In [ ]:
## Pull absolute temperature rasters from NetCDF for use in average temp calculation below
## Storing these in a dictionary so this only needs to be done once, improving processing time

nc_abs= arcpy.NetCDFFileProperties(in_nc_abs)
dim_abs = nc_abs.getDimensions()
top = nc_abs.getDimensionSize("time")

for i in range(0,top):  
    dim_value = nc_abs.getDimensionValue("time", i)
    abs_date = datetime.strptime(dim_value, '%m/%d/%Y')
    abs_month = abs_date.strftime("%B").lower()
    
    print(abs_date, abs_month)
    
    abs_raster_layer = arcpy.md.MakeNetCDFRasterLayer(in_nc_abs, "tem", "lon", "lat", "temp_abs_" + abs_month, '', "time " + dim_value, "BY_VALUE", "CENTER")
    abs_raster = arcpy.CopyRaster_management(abs_raster_layer, abs_raster_layer)
    abs_raster_dict[abs_month] = abs_raster
    

In [ ]:
count += 10  
obs_fields = [field.name for field in arcpy.ListFields(observations)]

try:
## Loop through observations that contain location data (Lat/Lon) and Year    
    for cnt, row in enumerate(arcpy.da.SearchCursor(observations,where_clause="Latitude is not NULL and Year_ is not null",
                                field_names=obs_fields)):
    
        unit_id = row[obs_fields.index('UnitID')]
## Select row (observation) and copy to in_memory layer        
        where = "UnitID = '" + unit_id + "'"
        observation_layer = arcpy.MakeFeatureLayer_management(observations, "observation_ml", where)
        observation = arcpy.CopyFeatures_management(observation_layer, "observation")

        if not row[obs_fields.index('Month_')]:
            print(cnt, unit_id, "Error: Null Month" )
            continue

        if not row[obs_fields.index('Year_')]:
            print(cnt, unit_id, "Error: Null Year")
            continue

        month = row[obs_fields.index('Month_')].strip()
        if not row[obs_fields.index('Day_')]:
            day = 15
        else:
            day = row[obs_fields.index('Day_')]
            
        year = row[obs_fields.index('Year_')]

        try:
            obs_date = datetime.strptime(month + "-" + str(day) + "-" + str(year), '%B-%d-%Y')
        except:
            print(cnt, unit_id, "strptime Error, ", month, day, year)
            continue
            
        obs_month = obs_date.strftime("%B").lower()
        nc_temp_date = str(nearest(datelist, obs_date).strftime('%m/%d/%Y %I:%M:%S %p'))
## Pull appropriate layer from NetCDF and copy to ESRI Raster layer
        try:
            temp_raster_layer = arcpy.md.MakeNetCDFRasterLayer(in_nc_temp, "temperature_anomaly", "longitude", "latitude", "temp_anomoly", '', "time " + "'" + nc_temp_date + "'", "BY_VALUE", "CENTER")
            temp_raster = arcpy.CopyRaster_management(temp_raster_layer, temp_raster_layer)
        except Exception as err:
            print(cnt, unit_id,"MakeNC: " + str(err))
            
## Calculate average temp by summing anomoly and absolute temp rasters
        try:
            calc_raster = arcpy.Raster(temp_raster) + arcpy.Raster(abs_raster_dict[obs_month])
            calc_raster.save('calc_raster_' + str(cnt+count))
        except Exception as err:
            print(cnt, unit_id, "Calc: " + str(err))

        raster_list = [[calc_raster, "avg_temp"]]
  
## Get temp value from calculated raster
        try:
            arcpy.sa.ExtractMultiValuesToPoints(observation, raster_list, "")    
        except Exception as err:
            print(cnt, unit_id,"Extract: " + str(err))


        if not arcpy.Exists(output_layer_location + "\\" + procname):
            if not procflag:
                procflag = True
                print("Create proc_layer: " + procname)
            else:
                procname = "proc_layer_temp_" + proc_datetime
                print("New proc layer_" + procname)   
            
            proc_layer = arcpy.CreateFeatureclass_management(output_layer_location, procname, template=observation)

## Append to proc_layer   
        try:
            arcpy.Append_management(observation, proc_layer)
        except:
            print(cnt, unit_id, "Append Error, trying again")
            time.sleep(2)
            try:
                arcpy.Append_management(observation, proc_layer)
            except Exception as err:
                print(cnt, unit_id,"Append: " + str(err))
            
            
        arcpy.Delete_management(observation)

except Exception as err:
    print(str(err))